# Place Reference 

## Background

When using lots of different data sets it is possible to end up with differing naming systems, for example using welsh
names rather than english for places in Wales or just places changing their names over time. To fix this we need to 
create a place reference, which will allow any name for a given place to be linked back to a single standardised name 
for our system. This means that we can merge in lots of different data-sets but maintain cohesion.

In this example we will be assuming we had data at a sub-unit level, and we want to create a link between our sub-units
and units over the time period. This often makes working with data simpler, as whilst you can and indeed must for your
base units provide a geographic id (GID), a number may not give you a good idea of where an error actually is placed
in your shapefile.

We will be using shapefiles from ReferenceCreation folder in the shapefile, which we set as a 'parish', and our
shapefiles we used in tutorial 1 as our 'districts'. We will be using parishes as base level here, in general you always
want to have the base level be the lowest level of separation you are using. You can have as many different match levels
as you want, but here we will just be using two. 

## Initialisation 

To start with we want to initialise our PlaceReference object, to do so we provide the working directory where all the
output files will be stored, as well as the name of the base level, your lowest level shapefile, and a list of all other
names; because you can submit multiple levels even if you are working with a single additional level it **must** be 
provided as a list!

In [1]:
from weightGIS import PlaceReference

ref_holder = PlaceReference("ExampleData/ReferenceOutput", "Parishes", ["Districts"])

## Linking places across time

We now want to link places across time. To do this we need to provide, as a string, a path to your lowest level 
shapefile directory; where all files within it are shapefiles you want to use as the lowest level base. This is our
*parishes* variable in this instance. Then we provide a list of other levels we want to relate to, in this case we want
to link to our districts so we provide a path to this as well. 

Finally we need to provide a list of lists, where each sub list is a list of indexes of the columns in the shapefile 
record's you want to use as a name. The list should be provided in the same order as your shapefiles, with the base 
being the first. If you want to use multiple columns for the name, provide more than one index per sub list. If the 
first column is not the GID in your base district, you will need to change this as well; in our case we don't need to
change this.

Once these are set it can run, you will see that we get some Ambiguous Relations. This is done on purpose, as for a 
unique reference to work each place can only have a single link to a single place. So parishes must be nested within 
districts etc. If you have data that does not allow for this, you will need to consider a different way of constructing 
your reference. Here we will assume that these ambiguous relations are from minor errors in the shapefile.

In [2]:
parishes = "ExampleData/ReferenceCreation"
districts = ["ExampleData/Shapefiles"]
indexes = [[1],
           [1]]

ref_holder.link_places_across_time(parishes, districts, indexes)


Processing 1931.shp contains 9 polygons

Processing 1951.shp contains 9 polygons
Ambiguous Relationship found for Denvialg in 1951 for ['Ecanlor', 'Danlhigh']
Ambiguous Relationship found for Nalmacn in 1951 for ['Ecanlor', 'Danlhigh']
Ambiguous Relationship found for Nebahekal in 1951 for ['Ecanlor', 'Danlhigh']
Please validate the 3 ambiguous relations before proceeding by creating a filecalled 'SetAmbiguous.csv' where there is now only a single relation for each ambiguous link


## Correcting for Ambiguity

To correct for ambiguity all you need to do is set a file, called SetAmbiguous.csv by default but you can call it 
something else as long as you tell the method its name, where each row has sets a one to one relation. So for example 
our first relation of Denvialg being related to Ecanlor	and Danlhigh in 1951, we will just set it to be part of Ecanlor.
The file should not have any file headers. Once we have made these corrections, we can create lists of all the unqiue 
names via write_linked_unique

In [3]:
ref_holder.write_linked_unique()

## Adding alternative names

In your datasets, external files, or just from shapefiles changing names you can now use these unique files to create
our Place Reference. The write_linked_unique will not over-ride files, this is because these edits are done manually 
due to often having to look up external names. **HOWEVER** if you are in a position to update Unique names via scripts 
then i would strongly recommend doing so! 

In this case i have edited unique districts to contain a alternative name for Ecanlor that was found in a hypothetical
dataset based on a differing language. When updating the unique, each additional column must contain the same name as 
the previous with a number on the end. For example if your base level is *Districts*, then the first column of 
alternative names for *Districts* would be *Districts1*. Once you have updated all the alternative names for your 
unique places you can construct the actual reference.

In [4]:
ref_holder.construct_reference()

## Output

What this has done, is allowed us to construct a reference that we can now us across all datasets that we want to 
weight. Any District called Ezandlor will be standardised back to Ecanlor and this means that the parish Ogardnep will
only ever have a single reference. For our actual weighting example, we only have a single set of external data that 
we don't need to standardise

|GID|Parish   |Districts|Alt_Districts_1|
|---|---------|---------|---------------|
|2  |Denvialg |Danlhigh |               |
|8  |Ogardnep |Ecanlor  |Ezandlor       |
|5  |Nebahekal|Ecanlor  |Ezandlor       |
|4  |Nalmacn  |Danlhigh |               |
|6  |Nidodog  |Ecanlor  |Ezandlor       |
|4  |Nalmacn  |Ecanlor  |Ezandlor       |
|2  |Denvialg |Ecanlor  |Ezandlor       |
|7  |Nirghol  |Nirghol  |               |
|3  |Dinram   |Danlhigh |               |